In [1]:
#from mpl_toolkits.basemap import Basemap
from matplotlib import pyplot as plt
import pandas as pd
import sklearn

In [2]:
data = pd.read_csv("property assessment 2022.csv")

differences between properties: 
neighborhood, price/quality correlations, correlations between features of property and price
segmentation, cluster analysis

In [3]:
data

,PID,CM_ID,GIS_ID,ST_NUM,ST_NAME,UNIT_NUM,CITY,ZIPCODE,BLDG_SEQ,NUM_BLDGS,...,KITCHEN_STYLE2,KITCHEN_STYLE3,HEAT_TYPE,HEAT_FUEL,AC_TYPE,FIRE_PLACE,ORIENTATION,NUM_PARKING,PROP_VIEW,CORNER_UNIT
0,100001000,,1.000010e+08,104 A 104,PUTNAM ST,NaN,EAST BOSTON,2128,1.0,1,...,S - Semi-Modern,S - Semi-Modern,W - Ht Water/Steam,NaN,N - None,0,NaN,3,A - Average,NaN
1,100002000,,1.000020e+08,197,LEXINGTON ST,NaN,EAST BOSTON,2128,1.0,1,...,M - Modern,M - Modern,F - Forced Hot Air,NaN,C - Central AC,0,NaN,0,A - Average,NaN
2,100003000,,1.000030e+08,199,LEXINGTON ST,NaN,EAST BOSTON,2128,1.0,1,...,S - Semi-Modern,S - Semi-Modern,S - Space Heat,NaN,N - None,0,NaN,0,A - Average,NaN
3,100004000,,1.000040e+08,201,LEXINGTON ST,NaN,EAST BOSTON,2128,1.0,1,...,S - Semi-Modern,S - Semi-Modern,W - Ht Water/Steam,NaN,N - None,0,NaN,0,A - Average,NaN
4,100005000,,1.000050e+08,203,LEXINGTON ST,NaN,EAST BOSTON,2128,1.0,1,...,S - Semi-Modern,NaN,W - Ht Water/Steam,NaN,N - None,0,NaN,0,A - Average,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178593,2205666000,,2.205666e+09,,KNOWLES ST,NaN,BRIGHTON,2135,1.0,1,...,NaN,NaN,NaN,NaN,NaN,0,NaN,0,NaN,NaN
178594,2205667000,,2.205667e+09,,LAKE ST,NaN,BRIGHTON,2135,1.0,1,...,NaN,NaN,NaN,NaN,NaN,0,NaN,0,NaN,NaN
178595,2205668000,,2.205668e+09,4,LAKE ST,NaN,BRIGHTON,2135,1.0,1,...,M - Modern,M - Modern,W - Ht Water/Steam,NaN,N - None,0,NaN,2,A - Average,NaN
178596,2205669000,,2.205669e+09,2193 2201,COMMONWEALTH AV,NaN,BRIGHTON,2135,1.0,1,...,NaN,NaN,NaN,NaN,NaN,0,NaN,0,NaN,NaN


In [ ]:
data.columns

Just a meaningless cell to play with the groupby function.

In [ ]:
#print(data.groupby("ZIPCODE").describe().columns)
data.groupby("ZIPCODE").agg(['sum'])

The following code drops the columns and data we absolutely don't need - such as IDs and duplicate addresses and mailing info.

In [6]:
cleaned = data.drop(['CM_ID', 'GIS_ID','UNIT_NUM','LUC','LU_DESC','CITY',"OWNER","MAIL_ADDRESSEE",'MAIL_ADDRESS', 'MAIL_CITY','MAIL_STATE', 'MAIL_ZIPCODE',"RES_UNITS","COM_UNITS","RC_UNITS","HEAT_FUEL","ORIENTATION"],axis=1)

Next, we need to create mappings for each categorical variable and fill in missing data in a reasonable way. For example, missing data for kitchens is equivalent to not having one at all.

mapping = {"a" : 1...}
df["col"] = df["col"].map(mapping)

This is an example of how to find the possible values in each column in order to figure out how to best fill in the data.

In [8]:
cleaned['STRUCTURE_CLASS'].unique()
#data.isnull().sum().to_string()
#for x in data.isnull().sum():
#  print(x)

array(['R - ', nan, 'D - Wood/Frame', 'C - Brick/Concr',
       'B - Reinf Concr', 'A - Struct Steel', 'E - Metal'], dtype=object)

In [ ]:
# drop properties without BLDG_VALUE
# replace nan in BLDG_TYPE with NoBld
# for GROSS_AREA and LIVING_AREA -
  # LIVING_AREA is often null for business type buildings, or condo management buildings
  # GROSS_AREA seems to just be a case of missing data?
  # this requires more investigation, but for now, dropping all properties with no gross area also appears to drop all those with
  # no living area either (meaning that all properties with no living area data also are missing gross area data)
# for kitchens - get the median type of kitchen and keep the count of kitchens rather than having 3 columns for three types of kitchens
# replace NAN in HEAT_FUEL with N - None
# same for HEAT_TYPE, AC_TYPE
# for FIRE_PLACE - simply assume nan = no fireplace
# for bathrooms, average out the style of bathroom
# merge res_floor and cd_floor
# for roof strucutre - change nan to "O - Other"
# for Structure class 
# for NUM_PARKING - assume there nan is no parking

Cleaning up the data

In [49]:
data[data["LAND_SF"].isnull() & data["UNIT_NUM"].isnull()].iloc[0].to_string()

'PID                                  100037000\nCM_ID                                         \nGIS_ID                             100037000.0\nST_NUM                               238 R238 \nST_NAME                           PRINCETON ST\nUNIT_NUM                                   NaN\nCITY                               EAST BOSTON\nZIPCODE                                   2128\nBLDG_SEQ                                   2.0\nNUM_BLDGS                                    2\nLUC                                        109\nLU                                          R3\nLU_DESC                   MULTIPLE BLDGS/1 LOT\nBLDG_TYPE                        CL - Colonial\nOWN_OCC                                      N\nOWNER                     CASIELLO JOHN M ETAL\nMAIL_ADDRESSEE            C/O MILDRED CASIELLO\nMAIL_ADDRESS                     529 WALNUT ST\nMAIL_CITY                               SAUGUS\nMAIL_STATE                                  MA\nMAIL_ZIPCODE                           

In [50]:
data.drop(data.index[data["GROSS_AREA"].isnull()]).isnull().sum().to_string()

'PID                     0\nCM_ID                   9\nGIS_ID                  8\nST_NUM                  0\nST_NAME                 0\nUNIT_NUM            80178\nCITY                    0\nZIPCODE                 0\nBLDG_SEQ                0\nNUM_BLDGS               0\nLUC                     0\nLU                      0\nLU_DESC                 0\nBLDG_TYPE             967\nOWN_OCC                 0\nOWNER                   0\nMAIL_ADDRESSEE     124352\nMAIL_ADDRESS           14\nMAIL_CITY               0\nMAIL_STATE              2\nMAIL_ZIPCODE           47\nRES_FLOOR           11016\nCD_FLOOR            86515\nRES_UNITS               0\nCOM_UNITS               0\nRC_UNITS                0\nLAND_SF             76524\nGROSS_AREA              0\nLIVING_AREA             0\nLAND_VALUE            176\nBLDG_VALUE            176\nTOTAL_VALUE           176\nGROSS_TAX             176\nYR_BUILT            10905\nYR_REMODEL          71820\nSTRUCTURE_CLASS     77846\nROOF_STRUCTURE      24454\n

In [ ]:
data.groupby("ZIPCODE")["TOTAL_VALUE"].mean()

TypeError: Could not convert 568,800518,000560,700962,20012,50012,100 to numeric

The prices are currently in string format with formatting commas in them. Those must be removed before trying any analysis

In [ ]:
data["TOTAL_VALUE"] = data["TOTAL_VALUE"].str.replace(',','')
data["TOTAL_VALUE"] = pd.to_numeric(data["TOTAL_VALUE"],errors='coerce')

In [ ]:
data["TOTAL_VALUE"]

0          719400.0
1          744800.0
2          730500.0
3          667900.0
4          742700.0
            ...    
178593     240500.0
178594      69000.0
178595    1252300.0
178596    2427000.0
178597    3213800.0
Name: TOTAL_VALUE, Length: 178598, dtype: float64

In [ ]:
data.groupby("ZIPCODE")["TOTAL_VALUE"].mean().sort_values()